# 📘 Systèmes de QA : RAG et RLHF

Dans ce notebook, nous présentons **deux techniques distinctes** pour la construction de systèmes de Question-Réponse (QA), chacune étant **indépendante** de l'autre. Elles illustrent deux approches différentes pour produire des réponses pertinentes à partir d’une question posée.



🔹 **Première partie – RAG (Retrieval-Augmented Generation)**  
Cette première partie introduit l’approche RAG, qui combine la récupération d’informations pertinentes à partir de documents avec la génération de texte.  
Le principe consiste à enrichir la réponse à une question en s’appuyant sur des passages extraits dynamiquement d’une source externe.



🔹 **Deuxième partie – RLHF (Reinforcement Learning with Human Feedback)**  
La deuxième partie explore une autre approche, basée sur l'apprentissage par renforcement guidé par des retours humains.  
Elle vise à affiner un modèle de génération pour qu’il produise des réponses plus alignées avec des attentes humaines, en s’appuyant sur des techniques d’optimisation via feedback.



✅ Ces deux approches sont traitées **séparément** dans ce notebook. Chacune a son propre pipeline, ses modèles et ses données.

➡️ Commençons maintenant par la première approche : **RAG (Retrieval-Augmented Generation)**.


# I-La technique RAG
## 🔧 Étapes principales du pipeline

1. Importation des bibliothèques nécessaires  
2. Extraction du texte depuis un fichier PDF  
3. Division du texte en paragraphes  
4. Création des représentations vectorielles (embeddings) 
5. Indexation vectorielle 
6. Recherche des passages pertinents  
7. Construction du contexte  
8. Génération de la réponse avec un modèle flan-T5-base


##  1. Importations des bibliothèques


In [ ]:
import fitz  
import re
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
from transformers import pipeline


##  2. Fonctions de traitement de texte
### 🔹 Extraction du texte depuis le PDF


In [ ]:
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    doc.close()
    return text


### 🔹 Découpage du texte en paragraphes


In [ ]:
def split_text_into_paragraphs(full_text, max_chars=200):
    paragraphs = []
    start = 0
    while start < len(full_text):
        end = start + max_chars
        if end < len(full_text):
            while end > start and full_text[end] not in [' ', '.', '\n']:
                end -= 1
            if end == start:
                end = start + max_chars
        paragraph = full_text[start:end].strip()
        paragraphs.append(paragraph)
        start = end
    return paragraphs


##  3. Fonctions pour l'encodage et l'indexation
### 🔹 Création des vecteurs


In [ ]:
def create_embeddings(paragraphs, embedding_model):
    return embedding_model.encode(paragraphs)


### 🔹 Création de l’index FAISS


In [ ]:
def create_faiss_index(embeddings):
    embeddings_np = np.array(embeddings).astype('float32')
    index = faiss.IndexFlatL2(embeddings_np.shape[1])
    index.add(embeddings_np)
    return index


### 🔹 Recherche dans l’index


In [ ]:
def search_faiss_index(query, index, embedding_model, k):
    query_embedding = embedding_model.encode([query])[0]
    scores, indices = index.search(np.array([query_embedding]).astype('float32'), k)
    return indices[0]


## 4. Génération de la réponse avec un modèle flan-T5-base


In [ ]:
def generate_answer(question, context, qa_pipeline):
    prompt = (
        f"Réponds à la question suivante en utilisant le contexte fourni.\n"
        f"Contexte : {context}\n"
        f"Question : {question}"
    )
    return qa_pipeline(prompt, max_new_tokens=150, truncation=True)[0]['generated_text']


# 5. Pipeline Principal
Voici l’exécution complète du système RAG.

In [ ]:
# === Pipeline Principal ===
if __name__ == "__main__":
        # 1. Spécifier le chemin vers le fichier PDF
    pdf_path = "C:/Users/fatim/OneDrive/Bureau/RAG_Project/définitions_générales.pdf"

    # 2. Extraction du texte et découpage
    full_text = extract_text_from_pdf(pdf_path)
    paragraphs = split_text_into_paragraphs(full_text)

    # Affichage du nombre de paragraphes extraits
    print(f"Nombre de paragraphes extraits : {len(paragraphs)}\n")

    # Affichage des premiers paragraphes extraits pour vérification
    print("Quelques paragraphes extraits :\n")
    for i, paragraph in enumerate(paragraphs[:5]):  # Afficher les 5 premiers paragraphes
        print(f"Paragraphe {i+1}: {paragraph}\n")

    # 3. Embedding
    embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
    embeddings = create_embeddings(paragraphs, embedding_model)

    # 4. Indexation
    index = create_faiss_index(embeddings)

    # 5. Recherche d'information
    query = "C'est quoi l'innovation ?"
    k = 3
    relevant_indices = search_faiss_index(query, index, embedding_model, k)
    relevant_indices = [int(i) for i in relevant_indices if i >= 0]

    print("La question posée : ",query)

    # Affichage des indices des passages pertinents
    print(f"Indices des passages pertinents : {relevant_indices}\n")

    # Affichage des passages pertinents
    print("Passages pertinents extraits :\n")
    for i in relevant_indices:
        print(f"Passage {i+1}: {paragraphs[i]}\n")

    # 6. Construction du contexte
    context = "\n".join([paragraphs[i] for i in relevant_indices])

    # Affichage du contexte
    print(f"Contexte utilisé pour générer la réponse : \n{context}\n")

    # 7. Génération de la réponse
    qa_pipeline = pipeline("text2text-generation", model="google/flan-t5-base")
    answer = generate_answer(query, context, qa_pipeline)

    # Affichage de la réponse générée
    print("\nRéponse générée :\n", answer)


# Resultat de l'exécution du pipeline principal :

Nombre de paragraphes extraits : 14

Quelques paragraphes extraits :

Paragraphe 1: L’intelligence artificielle (IA) est un ensemble de techniques permettant à des machines d’imiter
des fonctions cognitives humaines telles que l’apprentissage, le raisonnement, la résolution de

Paragraphe 2: problèmes ou la compréhension du langage.
L’ADN (acide désoxyribonucléique) est une molécule présente dans toutes les cellules vivantes et qui
contient les instructions génétiques nécessaires au

Paragraphe 3: développement et au fonctionnement des
organismes.
Le droit pénal est la branche du droit qui définit les infractions et détermine les peines
applicables aux personnes qui les commettent.
Le

Paragraphe 4: réchauffement climatique désigne l’augmentation progressive des températures moyennes à la
surface de la Terre, principalement causée par les activités humaines et les émissions de gaz à
effet de

Paragraphe 5: serre.
La démocratie est un système politique dans lequel le pouvoir est exercé par le peuple, soit
directement, soit par l’intermédiaire de représentants élus.
Un algorithme est une suite finie

La question posée :  C'est quoi l'innovation ?

Indices des passages pertinents : [11, 12, 10]

Passages pertinents extraits :

Passage 12: concepts.
L’innovation est l’introduction d’une nouveauté (produit, service, procédé ou organisation) qui
apporte une amélioration significative par rapport à l’existant.
Une récession est une baisse

Passage 13: prolongée de l’activité économique, généralement mesurée par une
diminution du PIB pendant au moins deux trimestres consécutifs.
Une machine virtuelle est un environnement logiciel qui simule un

Passage 11: cyberattaques.
L’abstraction en art désigne un style dans lequel les formes, les couleurs et les lignes ne
représentent pas directement la réalité, mais expriment des idées, des émotions ou des

Contexte utilisé pour générer la réponse :
concepts.
L’innovation est l’introduction d’une nouveauté (produit, service, procédé ou organisation) qui
apporte une amélioration significative par rapport à l’existant.
Une récession est une baisse
prolongée de l’activité économique, généralement mesurée par une
diminution du PIB pendant au moins deux trimestres consécutifs.
Une machine virtuelle est un environnement logiciel qui simule un
cyberattaques.
L’abstraction en art désigne un style dans lequel les formes, les couleurs et les lignes ne
représentent pas directement la réalité, mais expriment des idées, des émotions ou des

Device set to use cpu

Réponse générée :
 l’introduction d’une nouveauté (produit, service, procédé ou organisation) qui apporte une amélioration significative par rapport à l’existant 

# II- La technique RLHF

## 🔧 Étapes principales du pipeline
1. Importation des bibliothèques nécessaires    
2. Génération de la réponse (modèle / politique) 
3. Évaluation par retour humain (fonction de récompense)
4. Boucle d’entraînement (itérations/époques)  

##  1. Importations des bibliothèques

In [ ]:
import random

##  2.  Génération de la réponse (modèle / politique) 


In [ ]:
def model(input_text):
    responses = ["Bonjour !", "Salut !", "Je ne sais pas.", "Très bien."]
    return random.choice(responses)

##  3. 👤 Évaluation par retour humain (fonction de récompense)


In [ ]:
def human_feedback(response):
    good_responses = ["Bonjour !", "Très bien."]
    return 1 if response in good_responses else 0

## 4. 🔄 Boucle d’entraînement (itérations/époques)

In [ ]:
def train_model(epochs=10):
    for epoch in range(epochs):
        input_text = "Salut"  
        response = model(input_text)
        reward = human_feedback(response)
        print(f"Epoch {epoch+1}: Réponse: {response} | Récompense: {reward}")
        

train_model()

# 5. Pipeline Principal
Voici l’exécution complète du système RLHF.

In [ ]:
import random
def model(input_text):
    responses = ["Bonjour !", "Salut !", "Je ne sais pas.", "Très bien."]
    return random.choice(responses)
def human_feedback(response):
    good_responses = ["Bonjour !", "Très bien."]
    return 1 if response in good_responses else 0
def train_model(epochs=10):
    for epoch in range(epochs):
        input_text = "Salut"  
        response = model(input_text)
        reward = human_feedback(response)
        print(f"Epoch {epoch+1}: Réponse: {response} | Récompense: {reward}")
        

train_model()


# 6. Résultats

In [ ]:
PS C:\Users\USER> & "C:/Program Files/Python311/python.exe" c:/Users/USER/moncode.py
Epoch 1: Réponse: Je ne sais pas. | Récompense: 0
Epoch 2: Réponse: Salut ! | Récompense: 0        
Epoch 3: Réponse: Salut ! | Récompense: 0        
Epoch 4: Réponse: Salut ! | Récompense: 0        
Epoch 5: Réponse: Très bien. | Récompense: 1     
Epoch 6: Réponse: Bonjour ! | Récompense: 1      
Epoch 7: Réponse: Je ne sais pas. | Récompense: 0
Epoch 8: Réponse: Salut ! | Récompense: 0        
Epoch 9: Réponse: Très bien. | Récompense: 1     
Epoch 10: Réponse: Très bien. | Récompense: 1    
PS C:\Users\USER> 

Ce code simule de manière simple le principe du RLHF (Reinforcement Learning from Human Feedback) en générant aléatoirement des réponses à une entrée fixe ("Salut") à l’aide d’un faux modèle, puis en les évaluant à l’aide d’un retour humain simulé (une fonction qui attribue une récompense de 1 si la réponse est jugée "bonne", sinon 0). Lors de chaque itération, le modèle génère une réponse, reçoit un feedback, et affiche la récompense obtenue. Bien que le modèle ne s’améliore pas réellement au fil du temps (aucune mise à jour n’est effectuée), ce code illustre le principe fondamental du RLHF : produire une sortie, recevoir un retour humain, et utiliser ce retour pour guider l’apprentissage.